In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [5]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


model = Sequential()
model.add(Dense(512, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


1993 documents
91 classes ['agent.acquaintance', 'agent.age', 'agent.annoying', 'agent.answer_my_question', 'agent.bad', 'agent.be_clever', 'agent.beautiful', 'agent.birth_date', 'agent.boring', 'agent.boss', 'agent.busy', 'agent.can_you_help', 'agent.chatbot', 'agent.clever', 'agent.crazy', 'agent.fired', 'agent.funny', 'agent.good', 'agent.happy', 'agent.hobby', 'agent.hungry', 'agent.joke', 'agent.marry_user', 'agent.my_friend', 'agent.news', 'agent.occupation', 'agent.origin', 'agent.ready', 'agent.real', 'agent.residence', 'agent.right', 'agent.story', 'agent.sure', 'agent.talk_to_me', 'agent.there', 'agent.what_can_do', 'appraisal.bad', 'appraisal.good', 'appraisal.no_problem', 'appraisal.thank_you', 'appraisal.welcome', 'appraisal.well_done', 'confirmation.cancel', 'confirmation.no', 'confirmation.yes', 'dialog.hold_on', 'dialog.hug', 'dialog.i_do_not_care', 'dialog.sorry', 'dialog.what_do_you_mean', 'dialog.wrong', 'emotions.ha_ha', 'emotions.wow', 'greetings.bye', 'greetings.g

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Training data created
Epoch 1/200
399/399 [==============================] - 2s 2ms/step - loss: 4.3899 - accuracy: 0.0476
Epoch 2/200
399/399 [==============================] - 1s 2ms/step - loss: 3.6020 - accuracy: 0.1743
Epoch 3/200
399/399 [==============================] - 1s 2ms/step - loss: 2.9380 - accuracy: 0.2797
Epoch 4/200
399/399 [==============================] - 1s 2ms/step - loss: 2.2292 - accuracy: 0.4280
Epoch 5/200
399/399 [==============================] - 1s 2ms/step - loss: 1.7802 - accuracy: 0.5183
Epoch 6/200
399/399 [==============================] - 1s 2ms/step - loss: 1.3762 - accuracy: 0.6368
Epoch 7/200
399/399 [==============================] - 1s 2ms/step - loss: 1.1586 - accuracy: 0.6744
Epoch 8/200
399/399 [==============================] - 1s 2ms/step - loss: 0.8776 - accuracy: 0.7479
Epoch 9/200
399/399 [==============================] - 1s 2ms/step - loss: 0.7827 - accuracy: 0.7943
Epoch 10/200
399/399 [==============================] - 1s 2ms/step -